# KEP column generation

## Filter columns w.r.t constraints

In [10]:
function calculate_columns(node)
    node_pool=Array{Array{Int,1},1}()

    # colonne artificiel 
    push!(node_pool,vcat(1,ones(Int,nv(pool))))
    for k in 2:length(column_pool) 
        add=true
        
        # TO COMPLETE
        # There are two conditions under which the column should be eliminated:
        #  1. If the branching constraint x_ij<=0 applies and the edge (i,j) is involved in the cycle
        for v in tree[node].setzero
            # TO COMPLETE
            if column_pool[k][v+1] == 1
                add = false
            end
            if add==false break end
        end
        #  2. If the branching constraint x_ji>=1 apply and job i is not assigned to machine j
        for v in tree[node].setone
            # TO COMPLETE
            if column_pool[k][v+1] == 0
                add = false
            end
            if add==false break end
        end
        # TO COMPLETE 
        if add push!(node_pool,column_pool[k]) end
    end


    return node_pool
end

calculate_columns (generic function with 1 method)

## Translate the z solution to y solution

In [11]:
function calculate_ysol(mastersol,node_pool)
    
    y=Array{Float64}(undef,nv(pool))
    for i in 1:nv(pool)
        
        y[i]=sum(node_pool[c][i+1]*mastersol[c] for c in 1:length(mastersol))
        
    end
    
    return y
end

calculate_ysol (generic function with 1 method)

## Column generation

In [13]:
function Process_Node(nodeindex)
    println("\e[93m Processing node $nodeindex \e[00m")
    println("Variables set to zero are:", tree[nodeindex].setzero)
    println("Variables set to one are:", tree[nodeindex].setone)
    global nodestobedeleted = []
    
    # Calculate columns that are compatible with the branching constraints
    # Function calculate_columns TO COMPLETE (see above)
    node_pool = calculate_columns(nodeindex)
    
    # Recover the initial restricted master LP:
    # constraint and variable references as well as the model itself
    choosecerc,z,mastermodel = node_master(node_pool)
    
    while true
        # Solve the restricted master LP
        optimize!(mastermodel)
        
        node_infeasible = false
        
        # If the restricted master LP is infeasible then the node is infeasible
        if termination_status(mastermodel) == MOI.INFEASIBLE 
            println("\e[31m Node is infeasible \e[00m")
            return []
        end
        
        # Recover the optimal dual solution for the first set of constraints
        π = JuMP.dual.(choosecerc)
        
        
        # Recover the current master objective value
        value = JuMP.objective_value(mastermodel)
        
        # Extract the solution
        solution=[]
        for m in 1:length(z)
            push!(solution,JuMP.value.(z[m]))
        end
        solflat = collect(Iterators.flatten(solution))

        # Check whether the solution is integer
        # If so update the bounds
        if maximum(solflat-floor.(solflat)) <= ϵ
            # Found a feasible solution update the best known lower bound
            if value<=LB global LB=value end
            println("\e[32m Feasible solution with value $value found \e[00m")
            for i in 1:length(Queue)-1
                if tree[Queue[i]].ub <= LB
                    push!(nodestobedeleted,i)
                end
            end
        end
        
        # Update the node lower and upper bounds
        nodeub = sum(π[i] for i in 1:nv(pool))
        nodelb = value
        
        maxobj = 0

        # Solve the epk subproblem. 
        # Recover the objective value and the optimal pattern solution
        SPobj,column,x = kep_subproblem(π,nodeindex)

        if SPobj<Inf 
            # Extract the pattern from the solution
            pattern=findall(x->x!=0,column)
            # Calculate the reduced cost of the pattern   
            reduced_cost = SPobj

            if reduced_cost>=maxobj maxobj=reduced_cost end

            # If positive reduced cost pattern is found add it to the master
            if reduced_cost>0
                # Calculate the column's objective function contribution
                column_cost = sum(weight .* x)

                # Store column in the column pool
                push!(column_pool,vcat(column_cost,column))
                # Store column in the node pool
                push!(node_pool,vcat(column_cost,column))

                # ADD COLUMN TO MODEL
                # Add the new variable to the vector of variables for the machine
                push!(z,@variable(mastermodel,lower_bound=0,upper_bound=1))

                # Set the name of the new variable
                set_name(z[end], "z_$(length(column_pool))")

                # Set the coefficient of the variable in constraints choosecerc
                for j in 1:nv(pool)
                    set_normalized_coefficient(choosecerc[j], z[end], column[j])            
                end


                # Update the objective function    
                set_objective_function(mastermodel, objective_function(mastermodel) + column_cost*z[end])
                # ADD COLUMN TO MODEL

                println("\e[34m Pattern with items $pattern added \e[00m")
            end
            # Complete the upper bound calculation
            nodeub += SPobj
        else
            node_infeasible = true
            println("\e[31m Node is infeasible \e[00m")
            break
        end


        # If there is a subproblem that is infeasible then the node is infeasible RETURN EMPTY
        if node_infeasible return [] end

        # If upper bound is improved UPDATE
        if nodeub<=tree[nodeindex].ub tree[nodeindex].ub = nodeub end

        # If converged STOP
        if 2*abs((tree[nodeindex].ub-nodelb))/abs((nodelb+tree[nodeindex].ub))<ϵ || maxobj<=0
            println("Node relaxation is solved to optimality")
            println("Node lower bound is $nodelb")
            println("Node upper bound is $(tree[nodeindex].ub)")
            # If the artificial column is used in the final solution
            # then the master LP is infeasible

            if solution[1]>=ϵ
                node_infeasible = true
                println("\e[31m Node is infeasible \e[00m")
            end

            # If infeasible RETURN EMPTY
            if node_infeasible return []
            # If not infeasible RETURN SOLUTION   
            # Function calculate_ysol which translates the solution in terms of
            # z variables to a solution in terms of y variables: TO COMPLETE (See above) 
            else return calculate_ysol(solution,node_pool) end
        end
    end
end

LoadError: LoadError: UndefVarError: @variable not defined
in expression starting at In[13]:84